# 3. Attention メカニズムのコーディング

## 3. 3 Sefl-Attention を使って入力の異なる部分に注意を払う

In [1]:
import torch

In [2]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your x^1
   [0.55, 0.87, 0.66], # journey x^2
   [0.57, 0.85, 0.64], # starts x^3
   [0.22, 0.85, 0.33], # with x^4
   [0.77, 0.25, 0.10], # one x^5
   [0.05, 0.80, 0.55]  # step x^6
  ]
)

In [3]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 1.0783, 0.7070, 1.0865])


In [4]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1404, 0.2200, 0.2171, 0.1587, 0.1040, 0.1599])
Sum: tensor(1.0000)


In [5]:
def softmax_naive(x):
  return torch.exp(x) / torch.exp(x).sum(dim=0)

In [6]:
attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum()) 

Attention weights: tensor([0.1341, 0.2303, 0.2259, 0.1518, 0.1047, 0.1531])
Sum: tensor(1.)


In [7]:
attn_weight_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weight_2)
print("Sum:", attn_weight_2.sum())

Attention weights: tensor([0.1341, 0.2303, 0.2259, 0.1518, 0.1047, 0.1531])
Sum: tensor(1.0000)


In [8]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
  context_vec_2 += attn_weight_2[i] * x_i

print(context_vec_2)

tensor([0.4348, 0.6902, 0.5607])


In [9]:
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.5158, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 1.0783, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 1.0591, 0.7154, 1.0605],
        [0.5158, 1.0783, 1.0591, 0.8798, 0.4149, 0.8725],
        [0.4576, 0.7070, 0.7154, 0.4149, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.8725, 0.2935, 0.9450]])


In [10]:
attn_scores = inputs @ inputs.T

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.5158, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 1.0783, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 1.0591, 0.7154, 1.0605],
        [0.5158, 1.0783, 1.0591, 0.8798, 0.4149, 0.8725],
        [0.4576, 0.7070, 0.7154, 0.4149, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.8725, 0.2935, 0.9450]])


In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2088, 0.1996, 0.1971, 0.1287, 0.1214, 0.1444],
        [0.1341, 0.2303, 0.2259, 0.1518, 0.1047, 0.1531],
        [0.1347, 0.2296, 0.2254, 0.1514, 0.1074, 0.1516],
        [0.1212, 0.2128, 0.2087, 0.1745, 0.1096, 0.1732],
        [0.1512, 0.1940, 0.1956, 0.1448, 0.1861, 0.1283],
        [0.1339, 0.2111, 0.2057, 0.1705, 0.0955, 0.1833]])


In [12]:
row_2_sum = sum([0.1341, 0.2303, 0.2259, 0.1518, 0.1047, 0.1531])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 0.9999
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.8500, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [14]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4409, 0.6278, 0.5777],
        [0.4348, 0.6902, 0.5607],
        [0.4362, 0.6883, 0.5597],
        [0.4196, 0.6950, 0.5457],
        [0.4648, 0.6300, 0.5247],
        [0.4112, 0.6940, 0.5568]])


In [15]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4348, 0.6902, 0.5607])


## 3.4 訓練可能な重みをもつ Self-Attention を実装する

In [16]:
x_2 = inputs[1]
d_in = inputs.shape[1] # = 3
d_out = 2

In [17]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [18]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [19]:
keys = inputs @ W_key 
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [20]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [21]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.3863, 0.5577, 1.5440])


In [22]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
print(attn_weights_2)

tensor([0.1454, 0.2194, 0.2131, 0.1578, 0.0878, 0.1764])


In [23]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3175, 0.8298])


In [24]:
import torch.nn as nn 

class SelfAttention_v1(nn.Module):

  def __init__(self, d_in, d_out):
    super().__init__()
    self.W_query  = nn.Parameter(torch.rand(d_in, d_out))
    self.W_key    = nn.Parameter(torch.rand(d_in, d_out))
    self.W_value  = nn.Parameter(torch.rand(d_in, d_out))

  def forward(self, x):
    keys    = x @ self.W_key
    queries = x @ self.W_query
    values  = x @ self.W_value

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    context_vec = attn_weights @ values
    return context_vec



In [25]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.3120, 0.8167],
        [0.3175, 0.8298],
        [0.3173, 0.8292],
        [0.3107, 0.8140],
        [0.3058, 0.8030],
        [0.3115, 0.8157]], grad_fn=<MmBackward0>)


In [26]:
print(context_vec_2)

tensor([0.3175, 0.8298])


In [27]:
class SelfAttention_v2(nn.Module):
  
  def __init__(self, d_in, d_out, qkv_bias=False):
    super().__init__()
    self.W_query  = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key    = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value  = nn.Linear(d_in, d_out, bias=qkv_bias)

  def forward(self, x):
    keys    = self.W_key(x)
    queries = self.W_query(x)
    values  = self.W_value(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    context_vec = attn_weights @ values
    return context_vec
    

In [28]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0794,  0.0684],
        [-0.0801,  0.0677],
        [-0.0802,  0.0676],
        [-0.0817,  0.0655],
        [-0.0822,  0.0647],
        [-0.0810,  0.0664]], grad_fn=<MmBackward0>)


## 3. 5 Causal Attention で未来の単語を隠す

In [29]:
queries = sa_v2.W_query(inputs)
keys    = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
print(attn_weights)

tensor([[0.1936, 0.1659, 0.1664, 0.1487, 0.1734, 0.1521],
        [0.2059, 0.1673, 0.1677, 0.1422, 0.1679, 0.1490],
        [0.2053, 0.1673, 0.1677, 0.1425, 0.1678, 0.1493],
        [0.1935, 0.1680, 0.1681, 0.1496, 0.1656, 0.1552],
        [0.1838, 0.1675, 0.1676, 0.1554, 0.1665, 0.1591],
        [0.1947, 0.1674, 0.1677, 0.1486, 0.1677, 0.1539]],
       grad_fn=<SoftmaxBackward0>)


In [30]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [31]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1936, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2059, 0.1673, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2053, 0.1673, 0.1677, 0.0000, 0.0000, 0.0000],
        [0.1935, 0.1680, 0.1681, 0.1496, 0.0000, 0.0000],
        [0.1838, 0.1675, 0.1676, 0.1554, 0.1665, 0.0000],
        [0.1947, 0.1674, 0.1677, 0.1486, 0.1677, 0.1539]],
       grad_fn=<MulBackward0>)


In [32]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2849, 0.2473, 0.2475, 0.2203, 0.0000, 0.0000],
        [0.2186, 0.1992, 0.1993, 0.1849, 0.1980, 0.0000],
        [0.1947, 0.1674, 0.1677, 0.1486, 0.1677, 0.1539]],
       grad_fn=<DivBackward0>)


In [33]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[ 0.2899,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.4656,  0.1723,    -inf,    -inf,    -inf,    -inf],
        [ 0.4594,  0.1703,  0.1731,    -inf,    -inf,    -inf],
        [ 0.3396,  0.1394,  0.1404, -0.0242,    -inf,    -inf],
        [ 0.2183,  0.0874,  0.0882, -0.0185,  0.0786,    -inf],
        [ 0.3408,  0.1270,  0.1290, -0.0413,  0.1290,  0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [34]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2849, 0.2473, 0.2475, 0.2203, 0.0000, 0.0000],
        [0.2186, 0.1992, 0.1994, 0.1849, 0.1980, 0.0000],
        [0.1947, 0.1674, 0.1677, 0.1486, 0.1677, 0.1539]],
       grad_fn=<SoftmaxBackward0>)


In [37]:
# dropout
dropout = torch.nn.Dropout(p=0.5)
example = torch.ones(6, 6)
print(dropout(example))

tensor([[2., 0., 2., 2., 2., 2.],
        [2., 0., 2., 2., 0., 2.],
        [0., 2., 2., 0., 2., 0.],
        [2., 2., 2., 0., 0., 0.],
        [0., 0., 2., 2., 0., 2.],
        [0., 2., 2., 2., 0., 2.]])


In [40]:
torch.manual_seed(789)
print(dropout(attn_weights))

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4946, 0.4950, 0.4406, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3987, 0.0000, 0.0000, 0.0000],
        [0.3895, 0.3348, 0.0000, 0.0000, 0.3353, 0.0000]],
       grad_fn=<MulBackward0>)


In [42]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [50]:
class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()
    self.d_out = d_out
    self.W_query  = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key    = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value  = nn.Linear(d_in, d_out, bias=qkv_bias)

    self.dropout  = nn.Dropout(dropout)
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    keys    = self.W_key(x)
    queries = self.W_query(x)
    values  = self.W_value(x)

    attn_scores = queries @ keys.transpose(1, 2)
    attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)

    context_vec = attn_weights @ values
    return context_vec

In [51]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])
